In [1]:
from pathlib import Path
import numpy as np

In [2]:
path_to_data = Path("/media/nvme1/icare-data/features")
path_to_patient = path_to_data / "0484"

In [8]:
def get_unique_hour_files(path):
    all_files = sorted(list(path.iterdir()))
    seen_hours = set()
    unique_hour_files = []
    for file in all_files:
        hour = file.stem.split("_")[2]
        if hour not in seen_hours:
            unique_hour_files.append(file)
            seen_hours.add(hour)
        
    return unique_hour_files

In [15]:
files = get_unique_hour_files(path_to_patient)
hours = [int(f.stem.split("_")[2]) for f in files]

# Group the hours list with selected 6 hour epochs
def segment_hours_into_epochs(hour_list):
    # Initialize a list of 12 epochs with None
    epochs = [None] * 12
    
    start_idx = None
    current_epoch = None

    for idx, hour in enumerate(hour_list):
        # Determine which epoch the hour belongs to
        epoch_idx = hour // 6

        # If we're starting a new epoch or at the beginning of the list
        if current_epoch is None or current_epoch != epoch_idx:
            # If we have a starting index, this means we just finished an epoch
            if start_idx is not None:
                epochs[current_epoch] = (start_idx, idx - 1)
            start_idx = idx
            current_epoch = epoch_idx

    # Handle the case for the last hour in the list
    if start_idx is not None:
        epochs[current_epoch] = (start_idx, len(hour_list) - 1)
    
    return epochs

epoch_idx = segment_hours_into_epochs(hours)

In [16]:
def combine_arrays(arr_list):
    # Initialize the result array with nan values
    combined_arr = np.full((8, 72), np.nan)
    # Calculate the position to start inserting arrays
    start_col = 72 - (len(arr_list) * 12)
    
    for arr in arr_list:
        if arr.shape[1] < 12:
            padding_cols = 12 - arr.shape[1]
            arr = np.hstack([arr, np.full((8, padding_cols), np.nan)])
        
        # Insert into the combined array
        combined_arr[:, start_col:start_col+12] = arr
        start_col += 12
    
    return combined_arr

In [21]:
all_epoch_features = []
for epoch in epoch_idx:
    if epoch is not None:
        feature_files = files[epoch[0]:epoch[1]+1]
        features = []
        for feat_file in feature_files:
            feat = np.load(feat_file)
            features.append(feat)
    combined_features = combine_arrays(features)
    all_epoch_features.append(features)

In [20]:
len(all_epoch_features)

12